# motionLAB Annotations Data Format

If you wish to combine multiple datasets, it is often useful to convert them into a unified data format. 

Objective: This script will allow you to merge the annotations into motionLab format (COCO-style annotation file) containing Image IDs in your data.json (general) file.

Info: http://aidemos.cs.toronto.edu/nds/tutorial.html

## Data Structure

COCO format : https://cocodataset.org/#format-data ; https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch

In [ ]:
import json
import datetime

#### 1. INIT motionLAB JSON

In [ ]:
def init_json(file='mlab.json'):
    output = {
        "info": None,
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }
    
    output['info']= {
        "description": "Mixed Dataset",
        "url": "",
        "version": "1",
        "year": 2020,
        "date_created": datetime.datetime.utcnow().isoformat(' ')
    }

    
    
    with open(file, 'w') as f:
        json.dump(output,f)
    print("JSON INITIATED : {}".format(file))

init_json()

#### 2. load and Merge JSON

In [3]:
mlabjson=json.load(open('mlab.json'))
cocojson=json.load(open('cocoval2017.json'))
taojson=json.load(open('taotrainjson.json'))

In [4]:
def last_value(mlabjson, key="categories", subkey="id", initvalue=None):
    last_value=initvalue
    l=mlabjson[key]
    if l:
        last = mlabjson[key][-1]
        if last[subkey]: last_value=last[subkey] 
    return last_value
#TEST
last_value(mlabjson, key="categories", subkey="id", initvalue=0)

0

In [5]:
def merge_keys(mlabjson, newjson, key="images", indexed_key='image_id', root_dir=None, dir_key=['file_name', 'video']): 
    #NOTE: root_dir format str: 'root/path/'
    #NOTE: should update mlabjson and newjson
    
    #update newjson
    mlab_last_id = last_value(mlabjson, key=key, subkey="id", initvalue=0) #get last key value
    for ik,k in enumerate(newjson[key]): #update keys and values
        if isinstance(k, dict): #in case is "info": str or 'licenses' : ['unknown']
            original_id=k['id']
            new_id=mlab_last_id+(ik+1) 
            print('original id : {} > new id: {}'.format(original_id, new_id))
            #update newjson id keys
            newjson[key][ik]['id']=new_id  #update newjson id
            if indexed_key:  #update newjson indexed keys id (inside other newjson keys) 
                for nj_k in newjson: 
                    subs=0
                    if isinstance(newjson[nj_k], list) and isinstance(newjson[nj_k][0], dict) and newjson[nj_k][0].get(indexed_key, False):
                        for inj_v, nj_v in enumerate(newjson[nj_k]):
                                if nj_v[indexed_key]==original_id:
                                    newjson[nj_k][inj_v][indexed_key] = new_id
                                    subs+=1
                    print("{} id substitutions: {}".format(nj_k,subs))
            #additional specific updates
            if key=='images':  #update image filnemate root dir
                if root_dir:        
                    for dk in dir_key:
                        try:
                            newjson[key][ik][dk]=root_dir+newjson[key][ik][dk]
                        except:
                            pass 
                
    #update mlabjson
    mlabjson[key]=mlabjson[key]+newjson[key] #merge newjson : + or .extend()
    
    return mlabjson, newjson

##### Merging licenses (#WARNING update before images, videos, etc)

In [6]:
#TEST
key='licenses'
indexed_key='license'

print('\n>> COCO')
cocojson[key]=cocojson[key][0:2]
print(cocojson[key])
mlabjson, cocojson=merge_keys(mlabjson, cocojson, key=key, indexed_key=indexed_key)
print(cocojson[key])

print('\n>> TAO')
taojson[key]=taojson[key][0:2]
print(taojson[key])
mlabjson, taojson=merge_keys(mlabjson, taojson, key=key, indexed_key=indexed_key)
print(taojson[key])

print('\n>> mlabjson')
print(json.dumps(mlabjson[key], sort_keys=True, indent=4))


>> COCO
[{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}]
original id : 1 > new id: 1
info id substitutions: 0
licenses id substitutions: 0
images id substitutions: 1431
annotations id substitutions: 0
categories id substitutions: 0
original id : 2 > new id: 2
info id substitutions: 0
licenses id substitutions: 0
images id substitutions: 630
annotations id substitutions: 0
categories id substitutions: 0
[{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}]

>> TAO
['Unknown']
['Unknown']

>> mlabjson
[
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://cre

##### Merging Catagories

In [7]:
print(cocojson['categories'][0])
print(taojson['categories'][0])

{'supercategory': 'person', 'id': 1, 'name': 'person'}
{'frequency': 'r', 'id': 1, 'synset': 'acorn.n.01', 'image_count': 0, 'instance_count': 0, 'synonyms': ['acorn'], 'def': 'nut from an oak tree', 'name': 'acorn'}


In [8]:
#TEST
key='categories'
indexed_key='category_id'

print('\n>> COCO')
cocojson[key]=cocojson[key][0:2]
print(cocojson[key])
mlabjson, cocojson=merge_keys(mlabjson, cocojson, key=key, indexed_key=indexed_key)
print(cocojson[key])

print('\n>> TAO')
taojson[key]=taojson[key][0:2]
print(taojson[key])
mlabjson, taojson=merge_keys(mlabjson, taojson, key=key, indexed_key=indexed_key)
print(taojson[key])

print('\n>> mlabjson')
print(json.dumps(mlabjson[key], sort_keys=True, indent=4))


>> COCO
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}]
original id : 1 > new id: 1
info id substitutions: 0
licenses id substitutions: 0
images id substitutions: 0
annotations id substitutions: 11004
categories id substitutions: 0
original id : 2 > new id: 2
info id substitutions: 0
licenses id substitutions: 0
images id substitutions: 0
annotations id substitutions: 316
categories id substitutions: 0
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}]

>> TAO
[{'frequency': 'r', 'id': 1, 'synset': 'acorn.n.01', 'image_count': 0, 'instance_count': 0, 'synonyms': ['acorn'], 'def': 'nut from an oak tree', 'name': 'acorn'}, {'frequency': 'c', 'id': 2, 'synset': 'aerosol.n.02', 'image_count': 1, 'instance_count': 1, 'synonyms': ['aerosol_can', 'spray_can'], 'def': 'a dispenser that holds a substance under pressure', 'name': 'aerosol_can'}]
original id : 1 > ne

##### Merging images (licenses needs to be first updated)

In [9]:
#TEST
key='images'
indexed_key='image_id'

print('\n>> COCO')
cocojson[key]=cocojson[key][0:2]
print(cocojson[key])
root_dir='coco/images/val2017/'
dir_key=['file_name']
mlabjson, cocojson=merge_keys(mlabjson, cocojson, key=key, indexed_key=indexed_key, root_dir=root_dir, dir_key=dir_key)
print(cocojson[key])

print('\n>> TAO')
taojson[key]=taojson[key][0:2]
print(taojson[key])
root_dir='TAO/TAO_DIR/frames/'
dir_key=['file_name', 'video']
mlabjson, taojson=merge_keys(mlabjson, taojson, key=key, indexed_key=indexed_key, root_dir=root_dir, dir_key=dir_key)
print(taojson[key])

print('\n>> mlabjson')
print(json.dumps(mlabjson[key], sort_keys=True, indent=4))


>> COCO
[{'license': 4, 'file_name': '000000397133.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-14 17:02:52', 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg', 'id': 397133}, {'license': 1, 'file_name': '000000037777.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000037777.jpg', 'height': 230, 'width': 352, 'date_captured': '2013-11-14 20:55:31', 'flickr_url': 'http://farm9.staticflickr.com/8429/7839199426_f6d48aa585_z.jpg', 'id': 37777}]
original id : 397133 > new id: 1
info id substitutions: 0
licenses id substitutions: 0
images id substitutions: 0
annotations id substitutions: 19
categories id substitutions: 0
original id : 37777 > new id: 2
info id substitutions: 0
licenses id substitutions: 0
images id substitutions: 0
annotations id substitutions: 14
categories id substitutions: 0
[{'license': 4, 'file_name': 'coco/images/val2017/000000397133.jpg', 'co

##### Merging Annotations (#WARNING LAST to Merge, because it has the other ids)

In [10]:
#TEST
key='annotations'
indexed_key=None

print('\n>> COCO')
cocojson[key]=cocojson[key][0:2]
print(cocojson[key])
mlabjson, cocojson=merge_keys(mlabjson, cocojson, key=key, indexed_key=indexed_key)
print(cocojson[key])

print('\n>> TAO')
taojson[key]=taojson[key][0:2]
print(taojson[key])
mlabjson, taojson=merge_keys(mlabjson, taojson, key=key, indexed_key=indexed_key)
print(taojson[key])

print('\n>> mlabjson')
print(json.dumps(mlabjson[key], sort_keys=True, indent=4))


>> COCO
[{'segmentation': [[510.66, 423.01, 511.72, 420.03, 510.45, 416.0, 510.34, 413.02, 510.77, 410.26, 510.77, 407.5, 510.34, 405.16, 511.51, 402.83, 511.41, 400.49, 510.24, 398.16, 509.39, 397.31, 504.61, 399.22, 502.17, 399.64, 500.89, 401.66, 500.47, 402.08, 499.09, 401.87, 495.79, 401.98, 490.59, 401.77, 488.79, 401.77, 485.39, 398.58, 483.9, 397.31, 481.56, 396.35, 478.48, 395.93, 476.68, 396.03, 475.4, 396.77, 473.92, 398.79, 473.28, 399.96, 473.49, 401.87, 474.56, 403.47, 473.07, 405.59, 473.39, 407.71, 476.68, 409.41, 479.23, 409.73, 481.56, 410.69, 480.4, 411.85, 481.35, 414.93, 479.86, 418.65, 477.32, 420.03, 476.04, 422.58, 479.02, 422.58, 480.29, 423.01, 483.79, 419.93, 486.66, 416.21, 490.06, 415.57, 492.18, 416.85, 491.65, 420.24, 492.82, 422.9, 493.56, 424.39, 496.43, 424.6, 498.02, 423.01, 498.13, 421.31, 497.07, 420.03, 497.07, 415.15, 496.33, 414.51, 501.1, 411.96, 502.06, 411.32, 503.02, 415.04, 503.33, 418.12, 501.1, 420.24, 498.98, 421.63, 500.47, 424.39, 505.

# ANNOTATIONS FORMAT

## COCO

Annotation file format:

In [11]:
{
    "info": {info},
    "licenses": [license],
    "images": [image],
    "annotations": [annotation],
    "categories": [category], <-- Not in Captions annotations
    "segment_info": [segment] <-- Only in Panoptic annotations
}

SyntaxError: invalid syntax (<ipython-input-11-f6549ee7f2f8>, line 6)

In [ ]:
info{
    "year": int, 
    "version": str, 
    "description": str, 
    "contributor": str, 
    "url": str, 
    "date_created": datetime,
}
license{
    "id": int, 
    "name": str, 
    "url": str,
}
image{
    "id": int, 
    "width": int, 
    "height": int, 
    "file_name": str, 
    "license": int, "flickr_url": str, 
    "coco_url": str, "date_captured": datetime,
}
annotation{
    "id": int, 
    "image_id": int, 
    "category_id": int, 
    "segmentation": RLE or [polygon], 
    "area": float, 
    "bbox": [x,y,width,height], 
    "iscrowd": 0 or 1,
}

category{
    "id": int, 
    "name": str, 
    "supercategory": str,
}
segment{
    "id": int, 
    "category_id": int, 
    "area": int, 
    "bbox": [x,y,width,height], 
    "iscrowd": 0 or 1,
}


## TAO

Annotation file format:

In [ ]:
{
    "info" : info,
    "images" : [image],
    "videos": [video],
    "tracks": [track],
    "annotations" : [annotation],
    "categories": [category],
    "licenses" : [license],
}

In [ ]:
info: "like MS COCO"
image: {
    "id" : int,
    "video_id": int,
    "file_name" : str,
    "license" : int,
    # Redundant fields for COCO-compatibility
    "width": int,
    "height": int,
    "frame_index": int
}
video: {
    "id": int,
    "name": str,
    "width" : int,
    "height" : int,
    "neg_category_ids": [int],
    "not_exhaustive_category_ids": [int],
    "metadata": dict,  # Metadata about the video
}
track: {
    "id": int,
    "category_id": int,
    "video_id": int
}
category: {
    "id": int,
    "name": str,
    "synset": str,  # For non-LVIS objects, this is "unknown"
    ... [other fields copied from LVIS v0.5 and unused]
}
annotation: {
    "image_id": int,
    "track_id": int,
    "bbox": [x,y,width,height],
    "area": float,
    # Redundant field for compatibility with COCO scripts
    "category_id": int
}
license: {
    "id" : int,
    "name" : str,
    "url" : str,
}